In [3]:
from theano.sandbox import cuda
#cuda.use('gpu2')

In [4]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


## Setup

In [5]:
batch_size=64

In [6]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [7]:
X_test = np.expand_dims(X_test,1)
X_train = np.expand_dims(X_train,1)

In [8]:
X_train.shape

(60000, 1, 28, 28)

In [9]:
#y_train[:5]
len(y_train)

60000

In [10]:
y_train = onehot(y_train)
y_test = onehot(y_test)

In [11]:
y_train[:5]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [12]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

In [13]:
def norm_input(x): return (x-mean_px)/std_px

## Linear model

In [14]:
def get_lin_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [15]:
lm = get_lin_model()

In [16]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=64)
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [17]:
lm.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 12s - loss: 0.4328 - acc: 0.8715 - val_loss: 0.2964 - val_acc: 0.9158


In [18]:
lm.optimizer.lr=0.1

In [19]:
lm.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 12s - loss: 0.3003 - acc: 0.9142 - val_loss: 0.2841 - val_acc: 0.9195


In [20]:
lm.optimizer.lr=0.01

In [21]:
lm.fit_generator(batches, batches.N, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/4
60000/60000 [==============================] - 12s - loss: 0.2846 - acc: 0.9193 - val_loss: 0.2862 - val_acc: 0.9189
Epoch 2/4
60000/60000 [==============================] - 12s - loss: 0.2778 - acc: 0.9217 - val_loss: 0.2878 - val_acc: 0.9187
Epoch 3/4
60000/60000 [==============================] - 12s - loss: 0.2710 - acc: 0.9241 - val_loss: 0.2731 - val_acc: 0.9234
Epoch 4/4
60000/60000 [==============================] - 12s - loss: 0.2695 - acc: 0.9243 - val_loss: 0.2744 - val_acc: 0.9230


## Single dense layer

In [22]:
def get_fc_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(512, activation='softmax'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [23]:
fc = get_fc_model()

In [24]:
fc.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 13s - loss: 1.5633 - acc: 0.8627 - val_loss: 1.0374 - val_acc: 0.9107


In [25]:
fc.optimizer.lr=0.1

In [26]:
fc.fit_generator(batches, batches.N, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/4
60000/60000 [==============================] - 13s - loss: 0.7683 - acc: 0.9167 - val_loss: 0.5688 - val_acc: 0.9232
Epoch 2/4
60000/60000 [==============================] - 13s - loss: 0.4803 - acc: 0.9241 - val_loss: 0.4074 - val_acc: 0.9276
Epoch 3/4
60000/60000 [==============================] - 12s - loss: 0.3724 - acc: 0.9292 - val_loss: 0.3511 - val_acc: 0.9262
Epoch 4/4
60000/60000 [==============================] - 12s - loss: 0.3274 - acc: 0.9307 - val_loss: 0.3331 - val_acc: 0.9241


In [27]:
fc.optimizer.lr=0.01

In [28]:
fc.fit_generator(batches, batches.N, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/4
60000/60000 [==============================] - 13s - loss: 0.2978 - acc: 0.9341 - val_loss: 0.3174 - val_acc: 0.9271
Epoch 2/4
60000/60000 [==============================] - 12s - loss: 0.2783 - acc: 0.9369 - val_loss: 0.2861 - val_acc: 0.9326
Epoch 3/4
60000/60000 [==============================] - 13s - loss: 0.2631 - acc: 0.9379 - val_loss: 0.2874 - val_acc: 0.9295
Epoch 4/4
60000/60000 [==============================] - 13s - loss: 0.2505 - acc: 0.9400 - val_loss: 0.2761 - val_acc: 0.9339


## Basic 'VGG-style' CNN

In [29]:
def get_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [30]:
model = get_model()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [31]:
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 24s - loss: 0.1040 - acc: 0.9677 - val_loss: 0.0328 - val_acc: 0.9878


In [32]:
model.optimizer.lr=0.1

In [33]:
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 24s - loss: 0.0341 - acc: 0.9892 - val_loss: 0.0256 - val_acc: 0.9912


In [36]:
model.optimizer.lr=0.01

In [37]:
model.fit_generator(batches, batches.N, nb_epoch=8, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/8
60000/60000 [==============================] - 25s - loss: 0.0253 - acc: 0.9922 - val_loss: 0.0265 - val_acc: 0.9916
Epoch 2/8
60000/60000 [==============================] - 25s - loss: 0.0187 - acc: 0.9941 - val_loss: 0.0334 - val_acc: 0.9907
Epoch 3/8
60000/60000 [==============================] - 26s - loss: 0.0154 - acc: 0.9948 - val_loss: 0.0311 - val_acc: 0.9927
Epoch 4/8
60000/60000 [==============================] - 25s - loss: 0.0133 - acc: 0.9958 - val_loss: 0.0275 - val_acc: 0.9927
Epoch 5/8
60000/60000 [==============================] - 26s - loss: 0.0102 - acc: 0.9967 - val_loss: 0.0330 - val_acc: 0.9908
Epoch 6/8
60000/60000 [==============================] - 26s - loss: 0.0108 - acc: 0.9965 - val_loss: 0.0328 - val_acc: 0.9935
Epoch 7/8
60000/60000 [==============================] - 26s - loss: 0.0091 - acc: 0.9972 - val_loss: 0.0236 - val_acc: 0.9938
Epoch 8/8
60000/60000 [==============================] - 25s - loss: 0.0078 - acc: 0.9975 - val_loss: 0.0266 - 

## Data augmentation

In [38]:
model = get_model()

In [39]:
gen = image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(X_train, y_train, batch_size=64)
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [40]:
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 25s - loss: 0.2080 - acc: 0.9336 - val_loss: 0.0763 - val_acc: 0.9757


In [41]:
model.optimizer.lr=0.1

In [42]:
model.fit_generator(batches, batches.N, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/4
60000/60000 [==============================] - 26s - loss: 0.0706 - acc: 0.9778 - val_loss: 0.0577 - val_acc: 0.9821
Epoch 2/4
60000/60000 [==============================] - 25s - loss: 0.0547 - acc: 0.9827 - val_loss: 0.0620 - val_acc: 0.9809
Epoch 3/4
60000/60000 [==============================] - 25s - loss: 0.0488 - acc: 0.9848 - val_loss: 0.0428 - val_acc: 0.9856
Epoch 4/4
60000/60000 [==============================] - 25s - loss: 0.0442 - acc: 0.9866 - val_loss: 0.0427 - val_acc: 0.9865


In [43]:
model.optimizer.lr=0.01

In [44]:
model.fit_generator(batches, batches.N, nb_epoch=8, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/8
60000/60000 [==============================] - 25s - loss: 0.0390 - acc: 0.9881 - val_loss: 0.0367 - val_acc: 0.9890
Epoch 2/8
60000/60000 [==============================] - 25s - loss: 0.0364 - acc: 0.9886 - val_loss: 0.0360 - val_acc: 0.9881
Epoch 3/8
60000/60000 [==============================] - 25s - loss: 0.0347 - acc: 0.9892 - val_loss: 0.0319 - val_acc: 0.9896
Epoch 4/8
60000/60000 [==============================] - 26s - loss: 0.0327 - acc: 0.9897 - val_loss: 0.0376 - val_acc: 0.9887
Epoch 5/8
60000/60000 [==============================] - 25s - loss: 0.0322 - acc: 0.9903 - val_loss: 0.0292 - val_acc: 0.9901
Epoch 6/8
60000/60000 [==============================] - 25s - loss: 0.0286 - acc: 0.9909 - val_loss: 0.0370 - val_acc: 0.9888
Epoch 7/8
60000/60000 [==============================] - 24s - loss: 0.0288 - acc: 0.9909 - val_loss: 0.0281 - val_acc: 0.9914
Epoch 8/8
60000/60000 [==============================] - 24s - loss: 0.0274 - acc: 0.9913 - val_loss: 0.0227 - 

In [45]:
model.optimizer.lr=0.001

In [34]:
model.fit_generator(batches, batches.N, nb_epoch=14, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/14
60000/60000 [==============================] - 24s - loss: 0.0242 - acc: 0.9924 - val_loss: 0.0239 - val_acc: 0.9911
Epoch 2/14
60000/60000 [==============================] - 24s - loss: 0.0177 - acc: 0.9947 - val_loss: 0.0287 - val_acc: 0.9904
Epoch 3/14
60000/60000 [==============================] - 23s - loss: 0.0166 - acc: 0.9944 - val_loss: 0.0249 - val_acc: 0.9917
Epoch 4/14
60000/60000 [==============================] - 24s - loss: 0.0116 - acc: 0.9967 - val_loss: 0.0239 - val_acc: 0.9939
Epoch 5/14
60000/60000 [==============================] - 24s - loss: 0.0111 - acc: 0.9965 - val_loss: 0.0294 - val_acc: 0.9901
Epoch 6/14
60000/60000 [==============================] - 24s - loss: 0.0095 - acc: 0.9970 - val_loss: 0.0319 - val_acc: 0.9927
Epoch 7/14
60000/60000 [==============================] - 24s - loss: 0.0079 - acc: 0.9974 - val_loss: 0.0259 - val_acc: 0.9946
Epoch 8/14
60000/60000 [==============================] - 24s - loss: 0.0073 - acc: 0.9977 - val_loss: 0

In [35]:
model.optimizer.lr=0.0001

In [36]:
model.fit_generator(batches, batches.N, nb_epoch=10, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/10
60000/60000 [==============================] - 24s - loss: 0.0043 - acc: 0.9986 - val_loss: 0.0542 - val_acc: 0.9907
Epoch 2/10
60000/60000 [==============================] - 24s - loss: 0.0061 - acc: 0.9983 - val_loss: 0.0317 - val_acc: 0.9934
Epoch 3/10
60000/60000 [==============================] - 24s - loss: 0.0049 - acc: 0.9987 - val_loss: 0.0370 - val_acc: 0.9942
Epoch 4/10
60000/60000 [==============================] - 24s - loss: 0.0034 - acc: 0.9990 - val_loss: 0.0382 - val_acc: 0.9927
Epoch 5/10
60000/60000 [==============================] - 24s - loss: 0.0051 - acc: 0.9986 - val_loss: 0.0509 - val_acc: 0.9921
Epoch 6/10
60000/60000 [==============================] - 24s - loss: 0.0034 - acc: 0.9993 - val_loss: 0.0604 - val_acc: 0.9907
Epoch 7/10
60000/60000 [==============================] - 24s - loss: 0.0049 - acc: 0.9986 - val_loss: 0.0397 - val_acc: 0.9917
Epoch 8/10
60000/60000 [==============================] - 23s - loss: 0.0021 - acc: 0.9995 - val_loss: 0

## Batchnorm + data augmentation

In [37]:
def get_model_bn():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [38]:
model = get_model_bn()

In [39]:
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 62s - loss: 0.0833 - acc: 0.9743 - val_loss: 0.0274 - val_acc: 0.9904


In [40]:
model.optimizer.lr=0.1

In [41]:
model.fit_generator(batches, batches.N, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/4
60000/60000 [==============================] - 62s - loss: 0.0351 - acc: 0.9892 - val_loss: 0.0342 - val_acc: 0.9898
Epoch 2/4
60000/60000 [==============================] - 62s - loss: 0.0264 - acc: 0.9919 - val_loss: 0.0287 - val_acc: 0.9897
Epoch 3/4
60000/60000 [==============================] - 62s - loss: 0.0236 - acc: 0.9924 - val_loss: 0.0239 - val_acc: 0.9933
Epoch 4/4
60000/60000 [==============================] - 62s - loss: 0.0189 - acc: 0.9940 - val_loss: 0.0227 - val_acc: 0.9921


In [42]:
model.optimizer.lr=0.01

In [43]:
model.fit_generator(batches, batches.N, nb_epoch=12, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/12
60000/60000 [==============================] - 62s - loss: 0.0142 - acc: 0.9951 - val_loss: 0.0274 - val_acc: 0.9923
Epoch 2/12
60000/60000 [==============================] - 62s - loss: 0.0159 - acc: 0.9948 - val_loss: 0.0276 - val_acc: 0.9929
Epoch 3/12
60000/60000 [==============================] - 62s - loss: 0.0114 - acc: 0.9965 - val_loss: 0.0256 - val_acc: 0.9913
Epoch 4/12
60000/60000 [==============================] - 62s - loss: 0.0118 - acc: 0.9962 - val_loss: 0.0283 - val_acc: 0.9932
Epoch 5/12
60000/60000 [==============================] - 62s - loss: 0.0086 - acc: 0.9971 - val_loss: 0.0308 - val_acc: 0.9921
Epoch 6/12
60000/60000 [==============================] - 62s - loss: 0.0095 - acc: 0.9969 - val_loss: 0.0276 - val_acc: 0.9931
Epoch 7/12
60000/60000 [==============================] - 62s - loss: 0.0087 - acc: 0.9973 - val_loss: 0.0256 - val_acc: 0.9935
Epoch 8/12
60000/60000 [==============================] - 62s - loss: 0.0079 - acc: 0.9976 - val_loss: 0

In [44]:
model.optimizer.lr=0.001

In [45]:
model.fit_generator(batches, batches.N, nb_epoch=12, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/12
60000/60000 [==============================] - 62s - loss: 0.0066 - acc: 0.9980 - val_loss: 0.0360 - val_acc: 0.9918
Epoch 2/12
60000/60000 [==============================] - 62s - loss: 0.0074 - acc: 0.9978 - val_loss: 0.0167 - val_acc: 0.9952
Epoch 3/12
60000/60000 [==============================] - 62s - loss: 0.0034 - acc: 0.9989 - val_loss: 0.0298 - val_acc: 0.9930
Epoch 4/12
60000/60000 [==============================] - 61s - loss: 0.0037 - acc: 0.9988 - val_loss: 0.0323 - val_acc: 0.9924
Epoch 5/12
60000/60000 [==============================] - 61s - loss: 0.0036 - acc: 0.9988 - val_loss: 0.0304 - val_acc: 0.9940
Epoch 6/12
60000/60000 [==============================] - 62s - loss: 0.0046 - acc: 0.9986 - val_loss: 0.0267 - val_acc: 0.9937
Epoch 7/12
60000/60000 [==============================] - 62s - loss: 0.0040 - acc: 0.9988 - val_loss: 0.0293 - val_acc: 0.9939
Epoch 8/12
60000/60000 [==============================] - 62s - loss: 0.0037 - acc: 0.9989 - val_loss: 0

## Batchnorm + dropout + data augmentation

In [46]:
def get_model_bn_do():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [47]:
model = get_model_bn_do()

In [48]:
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 63s - loss: 0.1095 - acc: 0.9673 - val_loss: 0.0397 - val_acc: 0.9882


In [49]:
model.optimizer.lr=0.1

In [50]:
model.fit_generator(batches, batches.N, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/4
60000/60000 [==============================] - 62s - loss: 0.0449 - acc: 0.9861 - val_loss: 0.0275 - val_acc: 0.9917
Epoch 2/4
60000/60000 [==============================] - 63s - loss: 0.0349 - acc: 0.9886 - val_loss: 0.0367 - val_acc: 0.9886
Epoch 3/4
60000/60000 [==============================] - 62s - loss: 0.0299 - acc: 0.9904 - val_loss: 0.0300 - val_acc: 0.9908
Epoch 4/4
60000/60000 [==============================] - 63s - loss: 0.0292 - acc: 0.9906 - val_loss: 0.0248 - val_acc: 0.9927


In [51]:
model.optimizer.lr=0.01

In [52]:
model.fit_generator(batches, batches.N, nb_epoch=12, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/12
60000/60000 [==============================] - 62s - loss: 0.0233 - acc: 0.9929 - val_loss: 0.0200 - val_acc: 0.9937
Epoch 2/12
60000/60000 [==============================] - 63s - loss: 0.0241 - acc: 0.9924 - val_loss: 0.0234 - val_acc: 0.9928
Epoch 3/12
60000/60000 [==============================] - 62s - loss: 0.0215 - acc: 0.9934 - val_loss: 0.0212 - val_acc: 0.9936
Epoch 4/12
60000/60000 [==============================] - 62s - loss: 0.0180 - acc: 0.9949 - val_loss: 0.0275 - val_acc: 0.9921
Epoch 5/12
60000/60000 [==============================] - 61s - loss: 0.0139 - acc: 0.9954 - val_loss: 0.0202 - val_acc: 0.9937
Epoch 6/12
60000/60000 [==============================] - 63s - loss: 0.0160 - acc: 0.9947 - val_loss: 0.0368 - val_acc: 0.9910
Epoch 7/12
60000/60000 [==============================] - 62s - loss: 0.0141 - acc: 0.9957 - val_loss: 0.0225 - val_acc: 0.9940
Epoch 8/12
60000/60000 [==============================] - 63s - loss: 0.0123 - acc: 0.9962 - val_loss: 0

In [53]:
model.optimizer.lr=0.001

In [54]:
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 61s - loss: 0.0102 - acc: 0.9969 - val_loss: 0.0220 - val_acc: 0.9945


In [56]:
path = "data/mnist/"
model_path = path + 'models/'
model.save_weights(model_path+'cnn-mnist23-'+str(25)+'.pkl')

## Ensembling

In [60]:
X_test[0]

array([[[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   

In [61]:
y_test[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.])

In [62]:
model.load_weights(model_path+'cnn-mnist23-'+str(25)+'.pkl')

In [68]:
pred = model.predict(X_test,batch_size=256)

In [69]:
pred[1]

array([  3.7766e-11,   2.5412e-12,   1.0000e+00,   3.0006e-12,   1.9237e-14,   4.0677e-18,
         2.3784e-10,   9.4548e-14,   5.7713e-13,   6.8725e-16], dtype=float32)

In [70]:
y_test[1]

array([ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [67]:
def fit_model():
    model = get_model_bn_do()
    model.fit_generator(batches, batches.N, nb_epoch=1, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.N)
    model.optimizer.lr=0.1
    model.fit_generator(batches, batches.N, nb_epoch=4, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.N)
    model.optimizer.lr=0.01
    model.fit_generator(batches, batches.N, nb_epoch=12, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.N)
    model.optimizer.lr=0.001
    model.fit_generator(batches, batches.N, nb_epoch=18, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.N)
    return model

In [ ]:
models = [fit_model() for i in range(6)]

In [92]:
path = "data/mnist/"
model_path = path + 'models/'

In [93]:
for i,m in enumerate(models):
    m.save_weights(model_path+'cnn-mnist23-'+str(i)+'.pkl')

In [94]:
evals = np.array([m.evaluate(X_test, y_test, batch_size=256) for m in models])

 9984/10000 [============================>.] - ETA: 0s

In [95]:
evals.mean(axis=0)

array([ 0.016,  0.995])

In [96]:
all_preds = np.stack([m.predict(X_test, batch_size=256) for m in models])

In [97]:
all_preds.shape

(6, 10000, 10)

In [98]:
avg_preds = all_preds.mean(axis=0)

In [99]:
keras.metrics.categorical_accuracy(y_test, avg_preds).eval()

array(0.9969000220298767, dtype=float32)